The goal in this file is comparing the solution of regular Bounded-Makespan-Min-Cost with Fixed_x_Bounded_Makespan_Min_Cost and generate the figure automatically.

In [1]:
# importing packages
import numpy as np
import pandas as pd
from gurobipy import *

from scipy.stats import skewnorm

# from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
# from bokeh.plotting import figure, show
# from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS
# from bokeh.layouts import row, column, gridplot, widgetbox
# from bokeh.models.widgets import Panel, Tabs, Button

# from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import numpy as np
import pandas as pd

from gurobipy import *

# output_notebook()

In [2]:
ScnCount = [20, 50, 100, 200, 1000]
subScnCount = 20
maxScn = 1000

Number = range(1, 49)
SubNumber = [1]
directory2 = "./scenarioGameLeftSkew8/10_scenarioGame_" 

In [3]:
# reading data
def readData(readNumber, readSubNumber):  
        jobs = 30
        testNumber = readNumber # 1 to 48
        subTestNumber = readSubNumber # 1 to 10
        fileName = 'J' + str(jobs) + str(testNumber) + '_' + str(subTestNumber) + '.RCP'
        file = '../dataRCPSP/A' + str(jobs) + '/j' + str(jobs) + 'rcp/' + fileName
        my_cols = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
        df = pd.read_table(file, sep='\t',delim_whitespace=True, engine='python', names=my_cols)
        duration = df.iloc[2:, 0].values
        resourcesUsage = df.iloc[2:, 1:5].values
        successorsCount = df.iloc[2:, 5].values
        successors = df.iloc[2:, 6:9].values
        resourcesCount = int(df.iloc[0, 1])
        jobsCount = int(df.iloc[0,0])
        availableResources = df.iloc[1,0:resourcesCount].values
        return duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources

In [4]:
# model sets and parameters for FCT model
def FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, scenarioCount, subScenarioCount):  
        activities = np.array(range(jobsCount))
        resources = np.array(range(resourcesCount))
        scenarios = np.array(range(scenarioCount))
        subScenarios = np.array(range(subScenarioCount))
        T = duration.sum()
        d = dict(zip(activities,resourcesUsage))
        M = 150
        N = availableResources.max()
        return activities, resources, d, T, M , N, scenarios, subScenarios

In [5]:
# # scenario generating Uniform
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
#     aMin = 1 - (12**0.5)/10 #20%
#     aMax = 1 + (12**0.5)/10
#     for s in scenarios:
#         for a in activities:
#             cr = np.random.rand()
#             durationScenarios[a,s] = duration[a] * (aMin + cr * (aMax-aMin))
#     p = dict(zip(activities,durationScenarios))
#     return p



# # scenario generating Normal
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
#     for s in scenarios:
#         for a in activities:
#             if duration[a] != 0:
#                 cr = np.random.normal(duration[a], duration[a]*0.2)
#             else: cr = 0
#             if cr >= 0:
#                 durationScenarios[a,s] = cr
#             else: 
#                 durationScenarios[a,s] = duration[a]
#     p = dict(zip(activities,durationScenarios))
#     return p



# # scenario generating Left Skew 600
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
#     for a in activities[1:-1]:
#         rndsk = skewnorm.rvs(600, size=1000)
#         rndsk = rndsk * duration[a] *0.2 /rndsk.std()
#         rndsk = rndsk - rndsk.mean() + duration[a]
#         for s in scenarios:
#             durationScenarios[a,s] = rndsk[s]
    
#     p = dict(zip(activities,durationScenarios))
#     return p



# scenario generating Left Skew 8
def scenarioGen(scenarios, jobsCount, duration, activities):
    np.random.seed(0)
    durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
    for a in activities[1:-1]:
        rndsk = skewnorm.rvs(8, size=1000)
        rndsk = rndsk * duration[a] *0.1 /rndsk.std()
        rndsk = rndsk - rndsk.mean() + duration[a]
        for s in scenarios:
            durationScenarios[a,s] = rndsk[s]
    
    p = dict(zip(activities,durationScenarios))
    return p

In [6]:
# define variables of FCT model
def addFCTVars(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f, o, e

In [7]:
# define variables of FCT model
def addFCTVarsMM(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f

In [8]:
# define variables of FCT model
def addFCTVarsXX(activities, resources, T, availableResources):  
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return s, f, o, e

In [9]:
# constraint 2.5 of FCT model
def addFCTConst1(activities, successors, x):    
        FCT.addConstrs(
            (x[activityI, activityJ] 
             ==
             1
             for activityI in activities if activityI != activities[-1]
             for activityJ in activities[successors[activityI][successors[activityI] > 0].astype(np.int) - 1]), 
            name = "NetworkRelations")

In [10]:
# constraint 2.6a of FCT model
def addFCTConst2a(activities, x, s, p, M, scenarios):  
        FCT.addConstrs(
            (s[activityJ, scenario] -  s[activityI, scenario] 
             >=
             p[activityI][scenario] - M * (1 - x[activityI, activityJ])
             for scenario in scenarios
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [11]:
# constraint 2.6a of FCT model
def addFCTConst2aMM(activities, x, s, ppp, M):  
        FCT.addConstrs(
            (s[activityJ] -  s[activityI] 
             >=
             ppp[activityI] - M * (1 - x[activityI, activityJ])
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [12]:
# constraint 2.6b of FCT model
def addFCTConst2b(activities, s, o, scenarios):  
        FCT.addConstrs(
            (s[activity, scenario] 
             >=
             o[activity]
             for scenario in scenarios
             for activity in activities), 
            name = "NetworkStartTimeAndOrderRelations")

In [13]:
# constraint 2.7 of FCT model
def addFCTConst3(activities, x, f, resources, N):  
        FCT.addConstrs(
            (f[activityI, activityJ, resource] -  N * x[activityI, activityJ] 
             <=
             0
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]
             for resource in resources), 
            name = "NetworkFlowRelations")

In [14]:
# constraint 2.8 of FCT model
def addFCTConst4(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             d[activityI][resource]
             for activityI in activities if activityI != activities[0] 
             for resource in resources), 
            name = "OutgoingFlows")

In [15]:
# constraint 2.9 of FCT model
def addFCTConst5(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             d[activityJ][resource]
             for activityJ in activities if activityJ != activities[-1] 
             for resource in resources), 
            name = "IngoingFlows")

In [16]:
# constraint 2.10a of FCT model
def addFCTConst6(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activities[0], activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "FirstFlow")

In [17]:
# constraint 2.10b of FCT model
def addFCTConst7(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activityI, activities[-1], resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "LastFlow")

In [18]:
# constraint for bounding makespan
def addFCTConst9(activities, scenarios, scenarioCount, s, makespan):  
        FCT.addConstrs(
            (quicksum(s[activities[-1], scenario] for scenario in scenarios)
             <=
             makespan * scenarioCount
             for alaki in range(0,1)
             ), 
            name = "Makespandddd")

In [19]:
# Objective of FCT model -- min cost
def addFCTObj(activities, s, scenarios, scenarioCount, o): 
        obj = (quicksum(s[activity, scenario] - o[activity] 
                              for activity in activities
                              for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [20]:
# Objective of FCT model -- min makespan det
def addFCTObjMM(activities, s):
  
        obj = (s[activities[-1]])
        FCT.setObjective(obj, GRB.MINIMIZE)

In [21]:
# Objective of FCT model -- min expected makespan
def addFCTObjdW(activities, s, scenarios, scenarioCount):
  
        obj = (quicksum(s[activities[-1], scenario] for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [22]:
for i in Number:
    for j in SubNumber: 
        
        
        try:


            #MM##########################################################################################################
            ################################# MIN makespan for DETERMINISTIC
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN makespan for DETERMINISTIC    step=1  :   " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ############################################################################################################1

            makespanDet = 0
            makespanExp = 0

            duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources = readData(i, j)
            activities, resources, d, T, M , N, scenarios, subScenarios = \
                FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, maxScn, subScnCount)
            pAll = scenarioGen(scenarios, jobsCount, duration, activities)
            pMM = list(range(jobsCount))
            for act in range(jobsCount):
                pMM[act] = sum(pAll[act]) / maxScn
            FCT = Model('FCT: MIN makespan for DETERMINISTIC')
            FCT.setParam('TimeLimit', 60)
            x, s, f = addFCTVarsMM(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2aMM(activities, x, s, pMM, M)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjMM(activities, s)
            FCT.optimize()        

    #.............................................. Save MM-X

            directory = directory2 + str(i) + "_" + str(j) + "/"
            !mkdir $directory
            if FCT.Status == 2:

                # Open a file X
                xoName = directory+ "MM-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|MM|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()       


            #dW############################################################################################################
            ##################### MIN E(makespan) for SCENARIO base | XdW bede
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN E(makespan) for SCENARIO base | XdW bede   step=2  :  " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ##############################################################################################################2

            subP = dict(zip(activities,np.array(list(pAll.values()))[:,:subScnCount]))
            FCT = Model('FCT: MIN makespan for SCENARIO base')
            FCT.setParam('TimeLimit', 600)
            x, s, f, o, e = addFCTVars(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2a(activities, x, s, subP, M, subScenarios)
            addFCTConst2b(activities, s, o, subScenarios)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjdW(activities, s, subScenarios, subScnCount)
            FCT.optimize()        

    #.............................................. Save dW-X

            if FCT.Status == 2:
                # Open a file X
                xoName = directory + "dW-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            # Calculations
            if FCT.MIPGap <= 10000:
                aaa = 0 # Expected Makespan
                for vvv in range(subScnCount):
                    aaa += s[jobsCount - 1, vvv].X
            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|dW|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(aaa / subScnCount,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else: 
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()                 





            for scnCount in ScnCount:


                #SdW###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of dW shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  " + str(i) + "_" + str(j) + " || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################3

                fileName =  "dW-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)


                p = dict(zip(activities,np.array(list(pAll.values()))[:,:scnCount]))


                FCT = Model('for fixed X: MIN scenario used makespan | use x of dW')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName = "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SdW|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanExp = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



                #SMM###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of MM shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  " + str(i) + "_" + str(j) + "  || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################4

                fileName =   "MM-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)

                FCT = Model('for fixed X: MIN scenario used makespan | use x of MM')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SMM|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanDet = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



        except Exception as err:
            print("Erro {}".format(err))

iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_1_1
    MIN makespan for DETERMINISTIC    step=1  :   1_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 5109 rows, 5152 columns and 18495 nonzeros
Variable types: 4128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 4527 rows and 4669 columns
Presolve time: 0.01s
Presolved: 582 rows, 483 columns, 1742 nonzeros
Variable types: 281 continuous, 202 integer (202 binary)

Root relaxation: objective 3.800000e+01, 252 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   38.00000    0   10          -   38.00000      -   

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [1e+01, 2e+02]
  RHS range        [1e+00, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 20330 rows and 67475 columns
Presolve time: 0.08s
Presolved: 3630 rows, 1102 columns, 7821 nonzeros

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 4.999e+04
 Factor NZ  : 1.982e+05 (roughly 4 MBytes of memory)
 Factor Ops : 1.354e+07 (less than 1 second per iteration)
 Threads    : 22

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87661219e+02 -1.31344772e+03  1.94e+02 1.40e-03  3.29e+00     0s
   1   2.53945116e+02 -1.55806152e+03  5.47e+01 2.78e-16  7.15e-01     0s
   2   1.61532555e+02 -8.36466785e+02  4.02e+00 6.88e-15  2.04e-01     0s
   3   1.45300683e+02 -2.55472055e+02  2.88e+00 1.94e-15  8.09e-02     0s

Barrie

  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 5e-03]
  Bounds range     [1e+01, 2e+02]
  RHS range        [1e+00, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 165833 rows and 62689 columns
Presolve time: 0.70s
Presolved: 36867 rows, 11735 columns, 80071 nonzeros

Ordering time: 0.09s

Barrier statistics:
 AA' NZ     : 5.538e+05
 Factor NZ  : 2.125e+06 (roughly 40 MBytes of memory)
 Factor Ops : 1.526e+08 (less than 1 second per iteration)
 Threads    : 22

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.26687940e+02 -1.29954869e+04  1.74e+02 6.94e-18  2.57e+00     1s
   1   3.00421804e+02 -1.38860855e+04  6.34e+01 3.95e-03  6.82e-01     1s
   2   1.63271941e+02 -3.83111003e+03  2.86e+00 6.60e-15  8.09e-02     1s
   3   1.54755895e+02 -1.07692624e+03  2.18e-01 1.85e-15  2.30e-02     1s
   4   1.35948750e+02 -3.3

  11   4.40541704e+01  4.41512912e+01  7.42e-17 5.33e-08  5.46e-07    10s
  12   4.40813524e+01  4.40888191e+01  7.16e-17 4.21e-09  4.20e-08    10s
  13   4.40833269e+01  4.40842688e+01  7.41e-16 6.78e-10  5.30e-09    10s
  14   4.40833726e+01  4.40833808e+01  5.72e-16 2.63e-13  4.62e-11    10s
  15   4.40833795e+01  4.40833795e+01  1.11e-16 5.68e-14  2.84e-15    11s

Barrier solved model in 15 iterations and 10.56 seconds
Optimal objective 4.40833795e+01

Crossover log...

   21566 DPushes remaining with DInf 0.0000000e+00                11s
       0 DPushes remaining with DInf 3.2245318e-12                11s

       3 PPushes remaining with PInf 0.0000000e+00                11s
       0 PPushes remaining with PInf 0.0000000e+00                11s

  Push phase complete: Pinf 0.0000000e+00, Dinf 3.2245318e-12     11s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   20561    4.4083380e+01   0.000000e+00   0.000000e+00     11s
   20561    4.4083380e+01   0.000000e+00

  RHS range        [9e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 23928 rows and 68056 columns
Presolve time: 0.20s
Presolved: 32 rows, 72 columns, 140 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.800e+01
 Factor NZ  : 1.830e+02
 Factor Ops : 1.549e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.85502714e+01  3.16502714e+01  4.41e+01 0.00e+00  3.46e-01     0s
   1   3.85502714e+01  3.56671741e+01  5.80e+00 2.78e-17  5.03e-02     0s
   2   3.85502714e+01  3.79545987e+01  4.77e-01 4.16e-17  5.96e-03     0s
   3   3.85502714e+01  3.85083509e+01  3.76e-02 2.78e-17  3.89e-04     0s
   4   3.85502714e+01  3.85501991e+01  4.69e-05 3.26e-17  6.07e-07     0s
   5   3.85502714e+01  3.85502714e+01  4.73e-11 1.58e-17  6.12e-13     0s

Barrier solved mode

...

Optimize a model with 202700 rows, 68128 columns and 396980 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 5e-03]
  Bounds range     [2e+01, 2e+02]
  RHS range        [8e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 143891 rows and 60947 columns
Presolve time: 0.47s
Presolved: 58809 rows, 13179 columns, 125958 nonzeros

Ordering time: 0.08s

Barrier statistics:
 AA' NZ     : 1.182e+06
 Factor NZ  : 5.874e+06 (roughly 80 MBytes of memory)
 Factor Ops : 7.306e+08 (less than 1 second per iteration)
 Threads    : 22

Barrier performed 0 iterations in 0.78 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 5639 iterations and 0.78 seconds
Optimal objective  5.962214106e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_47_1
  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  47_1 || 1000
iiiiiii

       0 DPushes remaining with DInf 4.2970072e-12                 5s

     552 PPushes remaining with PInf 0.0000000e+00                 5s
       0 PPushes remaining with PInf 0.0000000e+00                 5s

  Push phase complete: Pinf 0.0000000e+00, Dinf 4.2970072e-12      5s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   20403    5.9409629e+01   0.000000e+00   0.000000e+00      6s
   20403    5.9409629e+01   0.000000e+00   0.000000e+00      7s

Solved with dual simplex
Solved in 61827 iterations and 6.97 seconds
Optimal objective  5.940962927e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
    MIN makespan for DETERMINISTIC    step=1  :   48_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 5129 rows, 5152 columns and 18515 nonzeros
Variable types: 4128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix ran

Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.47454134e+01  5.55204134e+01  8.30e+01 0.00e+00  1.56e-01     0s
   1   6.47454134e+01  5.95125580e+01  1.05e+01 4.16e-17  2.26e-02     0s
   2   6.47454134e+01  6.32076402e+01  7.38e-01 4.16e-17  2.77e-03     0s
   3   6.47454134e+01  6.45041798e+01  9.19e-02 2.78e-17  3.58e-04     0s
   4   6.47454134e+01  6.47384868e+01  2.37e-03 2.78e-17  9.16e-06     0s
   5   6.47454134e+01  6.47454108e+01  2.25e-08 1.39e-17  3.29e-09     0s
   6   6.47454134e+01  6.47454134e+01  2.94e-11 2.78e-17  7.01e-15     0s

Barrier solved model in 6 iterations and 0.11 seconds
Optimal objective 6.47454134e+01

Crossover log...

      21 DPushes remaining with DInf 0.0000000e+00                 0s
       0 DPushes remaining with DInf 0.0000000e+00                 0s

     209 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase comple

Optimal objective  6.370329781e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  48_1  || 100
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 103400 rows, 68128 columns and 204380 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [4e+01, 1e+02]
  RHS range        [8e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 103236 rows and 67734 columns
Presolve time: 0.14s
Presolved: 164 rows, 394 columns, 784 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.900e+02
 Factor NZ  : 1.903e+03
 Factor Ops : 2.601e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal 

   3  -1.07543969e+03  1.40689372e+02  4.60e-17 7.58e-02  1.51e-02     3s
   4   3.32889419e+01  1.40438663e+02  6.29e-18 9.68e-03  1.32e-03     3s
   5   1.71826312e+01  1.23428309e+02  6.72e-18 3.32e-03  1.31e-03     3s
   6   1.38941490e+01  9.81967088e+01  3.64e-17 1.07e-03  1.04e-03     3s
   7   4.26263495e+01  7.35435518e+01  1.13e-17 3.31e-04  3.81e-04     3s
   8   5.62952126e+01  6.64305680e+01  1.52e-17 9.41e-05  1.25e-04     3s
   9   6.30482273e+01  6.39864797e+01  9.43e-18 8.43e-06  1.16e-05     3s
  10   6.34805400e+01  6.35986951e+01  1.15e-17 1.67e-06  1.46e-06     3s
  11   6.34995077e+01  6.35052524e+01  2.67e-17 5.28e-08  7.08e-08     3s
  12   6.35008455e+01  6.35015840e+01  9.80e-17 7.81e-09  9.11e-09     3s
  13   6.35008516e+01  6.35014072e+01  1.60e-09 5.83e-09  6.85e-09     3s
  14   6.35008516e+01  6.35014071e+01  1.53e-09 5.83e-09  6.85e-09     3s
  15   6.35008516e+01  6.35014070e+01  1.55e-09 5.83e-09  6.85e-09     3s
  16   6.35008514e+01  6.35014068e+01 

In [23]:
print("FINISHED")

FINISHED
